In [1]:
import ROOT
import math

Welcome to JupyROOT 6.26/02


In [2]:
#Read the data in

inputFile =  ROOT.TFile("data.root")
tree = inputFile.Get("sig")

In [3]:
xmin = 0.0
xmax = 500.0

# Define the model

x = ROOT.RooRealVar("x", "x", xmin, xmax);
oneovertau = ROOT.RooRealVar("oneovertau", "oneovertau", -0.02, -0.1, -0.003)
sigma = ROOT.RooRealVar("sigma", "sigma", 5, 0.1, 10)
mass = ROOT.RooRealVar("mass", "mass",  0, 500)
nbkg = ROOT.RooRealVar("nbkg", "nbkg", 3000., 4000.)
nsig = ROOT.RooRealVar("nsig", "nsig", 0., 300.)
expo = ROOT.RooExponential("expo","exponential function", x, oneovertau)
gaus = ROOT.RooGaussian("gaus", "gaussian function", x, mass, sigma)
modelSB = ROOT.RooAddPdf("modelSB","modelSB", ROOT.RooArgList(gaus,expo), ROOT.RooArgList(nsig,nbkg))

In [4]:
# convert the data into a RooDataset
data = ROOT.RooDataSet("data","dataset with x", tree, ROOT.RooArgSet(x))
data.Print()

[#1] INFO:DataHandling -- RooAbsReal::attachToTree(x) TTree Float_t branch x will be converted to double precision.
[#1] INFO:DataHandling -- RooTreeDataStore::loadValues(data) Skipping event #52 because x cannot accommodate the value 506.767
[#1] INFO:DataHandling -- RooTreeDataStore::loadValues(data) Skipping event #176 because x cannot accommodate the value 914.034
[#1] INFO:DataHandling -- RooTreeDataStore::loadValues(data) Skipping event #340 because x cannot accommodate the value 556.639
[#1] INFO:DataHandling -- RooTreeDataStore::loadValues(data) Skipping event #360 because x cannot accommodate the value 549.012
[#1] INFO:DataHandling -- RooTreeDataStore::loadValues(data) Skipping ...
[#0] WARNING:DataHandling -- RooTreeDataStore::loadValues(data) Ignored 22 out-of-range events
RooDataSet::data[x] = 3957 entries


In [5]:
# Fit the model to the data
resSB = modelSB.fitTo(data, ROOT.RooFit.Save(True), ROOT.RooFit.Minos(True))

[#1] INFO:Minimization -- p.d.f. provides expected number of events, including extended term in likelihood.
[#1] INFO:Minimization -- RooAbsMinimizerFcn::setOptimizeConst: activating const optimization
[#1] INFO:Minimization --  The following expressions will be evaluated in cache-and-track mode: (gaus,expo)
 **********
 **    1 **SET PRINT           1
 **********
 **********
 **    2 **SET NOGRAD
 **********
 PARAMETER DEFINITIONS:
    NO.   NAME         VALUE      STEP SIZE      LIMITS
     1 mass         2.50000e+02  5.00000e+01    0.00000e+00  5.00000e+02
     2 nbkg         3.50000e+03  1.00000e+02    3.00000e+03  4.00000e+03
     3 nsig         1.50000e+02  3.00000e+01    0.00000e+00  3.00000e+02
     4 oneovertau  -2.00000e-02  8.50000e-03   -1.00000e-01 -3.00000e-03
     5 sigma        5.00000e+00  9.90000e-01    1.00000e-01  1.00000e+01
 **********
 **    3 **SET ERR         0.5
 **********
 **********
 **    4 **SET PRINT           1
 **********
 **********
 **    5 **SET STR

In [6]:
# plot results
canvas = ROOT.TCanvas()
canvas.cd()
xframe = x.frame(ROOT.RooFit.Title("expoRooFit.C"),ROOT.RooFit.Bins(100),ROOT.RooFit.Range(0.,500.))
data.plotOn(xframe)
modelSB.plotOn(xframe)
modelSB.plotOn(xframe, ROOT.RooFit.Components('expo'), ROOT.RooFit.LineColor(ROOT.kRed))
xframe.Draw()
ROOT.gPad.Update()

[#1] INFO:Plotting -- RooAbsPdf::plotOn(modelSB) directly selected PDF components: (expo)
[#1] INFO:Plotting -- RooAbsPdf::plotOn(modelSB) indirectly selected PDF components: ()


In [7]:
# now redo the steps by hand
# 1. construct the negative log likelihood 
nll = modelSB.createNLL(data) 
# 2. instantiate Minuit with that likelihood and minimize.
ROOT.RooMinuit(nll).migrad() 
# 3. do the profile likelihood in mll and plot
profile = nll.createProfile(nsig)
nsig.setVal(0)
print ("signal with",math.sqrt(2*profile.getVal()),"sigmas")

signal with 2.6112752998102526 sigmas
[#1] INFO:Minimization -- p.d.f. provides expected number of events, including extended term in likelihood.
 **********
 **   13 **MIGRAD        2500           1
 **********
 FIRST CALL TO USER FUNCTION AT NEW START POINT, WITH IFLAG=4.
 START MIGRAD MINIMIZATION.  STRATEGY  1.  CONVERGENCE WHEN EDM .LT. 1.00e-03
 FCN=-6756.93 FROM MIGRAD    STATUS=INITIATE       10 CALLS          11 TOTAL
                     EDM= unknown      STRATEGY= 1      NO ERROR MATRIX       
  EXT PARAMETER               CURRENT GUESS       STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  mass         3.81259e+02   2.71014e-01   1.27374e-03   6.40862e-02
   2  nbkg         3.95292e+03   6.19846e+01   3.35594e-01   4.66796e-05
   3  nsig         4.16521e+00   2.43437e+00   7.28384e-02  -9.19225e-03
   4  oneovertau  -9.89078e-03   1.73729e-04   6.97246e-03   4.73672e-03
   5  sigma        4.09130e-01   1.77067e-01   1.07709